In [ ]:
# AIRE Micro Tutor Identity Cell
import datetime as dt
learner_id = input('Enter your learner ID (email or handle): ').strip()
learner_role = input('Enter your learner role (e.g., researcher, educator): ').strip()
resource_id = 'notebook:tabular_basics'
session_id = f"{learner_id}-{dt.datetime.utcnow().isoformat()}"

In [ ]:
# AIRE Micro Tutor Imports
from prompt_tutor import evaluate_prompt
from aire_telemetry import log_event
from resources_map import RESOURCE_MAP

In [ ]:
import sys, subprocess, os
from pathlib import Path

# Colab Setup
if "google.colab" in sys.modules:
    print("Running in Google Colab. Installing dependencies...")
    subprocess.run(["pip", "install", "-q", "pandas", "numpy", "scikit-learn", "requests", "pydantic", "jsonschema", "plotly", "tqdm"])
    
    # Check for data
    if not (Path.cwd() / "data").exists():
        print("Data directory not found. Cloning repository...")
        subprocess.run(["git", "clone", "https://github.com/aire-program/aire-researcher-sandbox.git", "_repo"])
        
        # Move data and scripts to current directory
        if (Path("_repo/data").exists()):
            print("Moving data and scripts...")
            subprocess.run(["mv", "_repo/data", "."])
            subprocess.run(["mv", "_repo/scripts", "."])
            subprocess.run(["rm", "-rf", "_repo"])
        else:
            print("Warning: Data not found in cloned repo.")
    else:
        print("Data directory found.")


# Clean Synthetic Experiment Records

**What**: Load, validate, and clean synthetic experimental data.

**Why**: Data quality is the foundation of reliable research. Identifying missing values, outliers, and inconsistencies early prevents errors in downstream analysis.

**How**:
1. **Load the dataset** into a pandas DataFrame.
2. **Parse timestamps** to ensure correct temporal analysis.
3. **Perform range checks** to identify invalid data points.

**Key Concept**: **Data Validation** involves checking data against a set of rules (e.g., "metrics must be between 0 and 100") to ensure its logical consistency.

By the end of this notebook, you will have completed the listed steps and produced the outputs described in the success criteria.

### Success criteria
- You loaded experiment records.
- You parsed timestamps and checked ranges.
- You flagged or confirmed data quality.

In [ ]:
from pathlib import Path


def find_data_dir() -> Path:
    candidates = [Path.cwd() / "data", Path.cwd().parent / "data", Path.cwd().parent.parent / "data"]
    for candidate in candidates:
        if (candidate / "sample_texts" / "articles_sample.csv").exists():
            return candidate
    raise FileNotFoundError("data directory not found. Run scripts/generate_synthetic_data.py.")

DATA_DIR = find_data_dir()


In [ ]:
import pandas as pd

experiments = pd.read_csv(DATA_DIR / "sample_tabular" / "experiments_sample.csv")
experiments["timestamp"] = pd.to_datetime(experiments["timestamp"])

print("Dataset shape", experiments.shape)
print("Missing values", experiments.isna().sum())
experiments.head()


## Basic range checks

In [ ]:
metric_out_of_range = (~experiments["metric_value"].between(0, 100)).sum()
print(f"Records outside expected metric range: {metric_out_of_range}")
experiments.describe()


### If you get stuck / What to try next

If you get stuck: ensure timestamps parse by rerunning the cleaning cell; confirm data generation. What to try next: create features in pipelines/tabular/feature_engineering.ipynb and visualize them in the tabular notebooks.

In [ ]:
# AIRE Micro Tutor Prompt + Feedback Cell
from aire_llm_client import chat_completion

def call_llm(prompt_text: str) -> str:
    messages = [
        {'role': 'system', 'content': 'You are an AI helper responding concisely.'},
        {'role': 'user', 'content': prompt_text},
    ]
    return chat_completion(messages)

prompt_text = input('Enter the prompt you want to test: ').strip()
ai_response = call_llm(prompt_text)
evaluation = evaluate_prompt(prompt_text, learner_role)

clarity = evaluation.get('clarity_score')
context = evaluation.get('context_score')
constraints = evaluation.get('constraints_score')
evaluation_score = evaluation.get('evaluation_score')
primary_weakness = evaluation.get('primary_weakness', '')
recommended_resource_id = RESOURCE_MAP.get(primary_weakness) or RESOURCE_MAP.get('evaluation')

log_event(
    event_name='micro_tutor_evaluation',
    user_id=learner_id or session_id,
    metadata={
        'resource_id': resource_id,
        'learner_role': learner_role,
        'clarity': clarity,
        'context': context,
        'constraints': constraints,
        'evaluation_score': evaluation_score,
        'primary_weakness': primary_weakness,
    },
)

print('\n--- AI Response ---')
print(ai_response)
print('\n--- Feedback ---')
print(evaluation.get('summary', 'No summary provided.'))
print('Primary weakness:', primary_weakness or 'n/a')
print('Strengths:', ', '.join(evaluation.get('strengths', [])) or 'n/a')
print('Suggestions:', ', '.join(evaluation.get('suggestions', [])) or 'n/a')
print('\nRecommended resource:', recommended_resource_id or 'n/a')